In [1]:
from __future__ import print_function

import os
import sys
import glob
import time
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf


In [2]:
def import_dataset(ticker, start_date, end_date):

    dataset = yf.download(ticker, start=start_date, end=end_date, interval='1d')

    print("num rows",dataset.shape[0])
    # pd.set_option('display.max_rows', None)
    # pd.set_option('display.max_columns', None)
    # pd.set_option('display.width', None)
    # pd.set_option('display.max_colwidth', None)
    dataset = dataset.dropna().dropna()
    dataset.dropna(how='any', inplace=True)
    print("Num rows for df Close col",len(dataset['Close']))
    print(dataset.columns)
    dataset = dataset.reset_index()
    #reorder to split the data to train and test
    desired_order = ['Date','Open', 'Close', 'High', 'Low']
    if 'Date' in dataset.columns:
        dataset = dataset[desired_order]
    else:
        print("Column 'Date' is missing.")
    dataset = dataset.set_index('Date')
    print(dataset.head())
    print("day count",dataset.index.max()-dataset.index.min())

    #print(dataset.iloc[521],dataset.iloc[522],dataset.iloc[523])
    # pd.reset_option('display.max_rows')
    # pd.reset_option('display.max_columns')
    # pd.reset_option('display.width')
    # pd.reset_option('display.max_colwidth')

    return dataset

In [3]:
def compare_stocks(index_ticker, stock_ticker, stock_dataset, start_date, end_date):
    index_data = yf.download(index_ticker, start=start_date, end=end_date, interval='1d')

    stock_data = stock_dataset.dropna()
    index_data = index_data.dropna()

    stock_data = stock_data[stock_data.index <= end_date]
    index_data = index_data[index_data.index <= end_date]

    #rebase
    stock_rebased = stock_data / stock_data.iloc[0] * 100
    index_rebased = index_data / index_data.iloc[0] * 100

    fig, axs = plt.subplots(2, 2, figsize=(15, 12))

    axs[0, 0].plot(stock_rebased.index, stock_rebased['Open'], label=f'{stock_ticker} Open Price', color='g')
    axs[0, 0].plot(stock_rebased.index, stock_rebased['Close'], label=f'{stock_ticker} Close Price', color='m')
    axs[0, 0].plot(index_rebased.index, index_rebased['Close'], label=f'{index_ticker} Close Price', color='m', linestyle='--')
    axs[0, 0].set_title('Open and Close Prices')
    axs[0, 0].legend()
    axs[0, 0].grid(True)

    axs[0, 1].plot(stock_rebased.index, stock_rebased['Close'], label=f'{stock_ticker} Close Price', color='m')
    axs[0, 1].plot(stock_rebased.index, stock_rebased['High'], label=f'{stock_ticker} High Price', color='b')
    axs[0, 1].plot(index_rebased.index, index_rebased['Close'], label=f'{index_ticker} Close Price', color='m', linestyle='--')
    axs[0, 1].set_title('Close and High Prices')
    axs[0, 1].legend()
    axs[0, 1].grid(True)

    axs[1, 0].plot(stock_rebased.index, stock_rebased['Close'], label=f'{stock_ticker} Close Price', color='m')
    axs[1, 0].plot(stock_rebased.index, stock_rebased['Low'], label=f'{stock_ticker} Low Price', color='r')
    axs[1, 0].plot(index_rebased.index, index_rebased['Close'], label=f'{index_ticker} Close Price', color='m', linestyle='--')
    axs[1, 0].set_title('Close and Low Prices')
    axs[1, 0].legend()
    axs[1, 0].grid(True)

    axs[1, 1].plot(stock_rebased.index, stock_rebased['High'], label=f'{stock_ticker} High Price', color='b')
    axs[1, 1].plot(stock_rebased.index, stock_rebased['Low'], label=f'{stock_ticker} Low Price', color='r')
    axs[1, 1].plot(index_rebased.index, index_rebased['High'], label=f'{index_ticker} High Price', color='b', linestyle='--')
    axs[1, 1].set_title('High and Low Prices')
    axs[1, 1].legend()
    axs[1, 1].grid(True)

    plt.tight_layout()
    plt.show()